In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import avg, col, when, row_number
from pyspark.sql.window import Window

In [3]:
#docker inspect -f '{{range.NetworkSettings.Networks}}{{.IPAddress}}{{end}}' spark_spark_1

with open("remote_ip.txt") as f:
    scheduler_ip = f.read()

## GCP
Currently uses local pyspark to connect to my existing GCP resources. Jobs can be submitted to container cluster (only cloud storage right now) but is slower due to small data volume.

In [32]:
spark = (
    SparkSession.builder.appName("gcp")
    #.config("spark.master", f"spark://{scheduler_ip}:7077")
    .config("spark.jars", "spark-3.3-bigquery-0.32.2.jar,gcs-connector-hadoop3-latest.jar")
    .config("spark.master", "local[*]")
    .getOrCreate()
)

spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile","image_build/gcs_keyfile.json")
#spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile","/opt/bitnami/spark/gcs_keyfile.json")

# Allegedly needed to storage actions but doesn't populate anything
bucket = "dataproc-temp-us-east1-49528533322-xcoaaowy"
spark.conf.set('temporaryGcsBucket', bucket)

In [33]:
df = spark.read.csv("gs://mark-dbt-learn-bucket/Worlds_Best_Employers.csv", header=True)
df.show(5)

+----+-------------------+--------------------+-----------------+---------+
|RANK|               NAME|          INDUSTRIES|COUNTRY/TERRITORY|EMPLOYEES|
+----+-------------------+--------------------+-----------------+---------+
|  1.|Samsung Electronics|Semiconductors, E...|       South Kore|  266,673|
|  2.|          Microsoft|IT, Internet, Sof...|    United States|  221,000|
|  3.|                IBM|Semiconductors, E...|    United States|  250,000|
|  4.|           Alphabet|IT, Internet, Sof...|    United States|  156,500|
|  5.|              Apple|Semiconductors, E...|    United States|  154,000|
+----+-------------------+--------------------+-----------------+---------+
only showing top 5 rows



In [34]:
# Throws an error when connecting to container cluster
raw_claims_df = spark.read.format("bigquery") \
  .option("parentProject", "methodical-aura-373904") \
  .option("dataset", "comp653") \
  .option("table", "raw_insurance_claims") \
  .load()

raw_claims_df.show(2)

+------------------+---+-------------+----------------+------------+----------+-----------------+---------------------+--------------+-----------+-----------+-----------------------+------------------+---------------+--------------------+-------------+------------+-------------+-------------+--------------+-----------------+---------------------+--------------+-------------+------------------+------------------------+---------------------------+---------------+---------------+---------+-----------------------+------------------+------------+--------------+-------------+---------+----------+---------+--------------+----+
|months_as_customer|age|policy_number|policy_bind_date|policy_state|policy_csl|policy_deductable|policy_annual_premium|umbrella_limit|insured_zip|insured_sex|insured_education_level|insured_occupation|insured_hobbies|insured_relationship|capital_gains|capital_loss|incident_date|incident_type|collision_type|incident_severity|authorities_contacted|incident_state|incident_

In [35]:
#Stop the Spark session
spark.stop()